# Timer Class - 19.8 Takeaway

Develop a timer class that manages the execution of deferred tasks. The timer constructor takes as its argument an object which includes a run method and a string valued name field. The class must support

- starting a thread, identified by name, at a given time in the future
- canceling a thread, identified by name (the cancel request is to be ignored if the thread has already started).

## Brainstorming

We need a heap and a hash table to keep track of the tasks.

The min heap is perfect for ordering the tasks by time and popping accordingly. Entries into the heap should be a tuple with this value:

`entry = (launch_time, task_id, task)`


With the hashtable, we can keep correlate the task_id with the entries that pop out of the heapq. It's perfect for cancelling tasks that haven't been scheduled yet, because we can easily use a `del` operation on a hastable key-value pair, and then when we pop the entry out of the queue, if it's not in the hash table, we can assume the task was cancelled and skip the run of the task.


We also use a simple 1 counter semaphore for all reads and writes into the hash table and heap queue.

In [52]:
import heapq
import threading
import time

class Timer:
    def __init__(self):
        self.task_heap = []     # min-heap of tasks
        self.task_table = {}    # task table, maps task ids to entries in the heap
        self.semaphore = threading.Semaphore(1)  # semaphore to control access to the shared data structures
        self._stop = False      # flag to indicate whether the dispatch thread should stop
    
    def add_task(self, task_id, launch_time, task):
        """Add a task to the timer."""
        entry = (launch_time, task_id, task)
        with self.semaphore:
            heapq.heappush(self.task_heap, entry)
            self.task_table[task_id] = entry
            if entry == self.task_heap[0]:
                self.semaphore.release()
                return True
        # self.semaphore.release()
        return False
    
    def cancel_task(self, task_id):
        """Cancel a task from the timer."""
        with self.semaphore:
            if task_id in self.task_table:
                del self.task_table[task_id]
                return True
        return False
    
    def dispatch_thread(self):
        """Thread that dispatches tasks at their scheduled launch time."""
        while not self._stop:
            with self.semaphore:
                while self.task_heap and self.task_heap[0][0] <= time.time():
                    entry = heapq.heappop(self.task_heap)
                    task_id, task = entry[1], entry[2]
                    
                    # If task id isn't here, it was cancelled. Skip the run
                    if task_id not in self.task_table:
                        continue

                    if task:
                        task.run()
                    if task_id in self.task_table:
                        del self.task_table[task_id]
                if not self.task_heap:
                    self.semaphore.release()
                    break
                else:
                    remaining_time = self.task_heap[0][0] - time.time()
                    self.semaphore.release()
                    time.sleep(remaining_time)
    
    def start(self):
        """Start the dispatch thread."""
        self.dispatch_thread = threading.Thread(target=self.dispatch_thread)
        self.dispatch_thread.start()
    
    def stop(self):
        """Stop the dispatch thread."""
        #self._stop = False
        self.dispatch_thread.join()

In [54]:
# Let's test this class!

class PrintTask:
    def __init__(self, message):
        self.message = message
    
    def run(self):
        print(self.message)

timer = Timer()

# Typical format for tasks
task_id = 1
launch_time = time.time() + 5  # launch the task after 5 seconds

timer.add_task(task_id, launch_time, PrintTask("This msg could get cancelled!"))
timer.add_task(2, time.time() + 2, PrintTask("Hello, world!")) # Scheduling another task 2 seconds from now

timer.start()
timer.cancel_task(1) # Try commenting this line out to see the difference in behavior. Will cancel task_id 1 before it's scheduled

# Wait for the dispatch thread to complete
timer.stop()

Hello, world!
